In [1]:
import cv2
import json
import os
import os.path as osp
from pathlib import Path
from tqdm import tqdm
from time import time

from core import ObjectDetector, CornerDetector

In [2]:
#dt_vehicle = ObjectDetector('yolov4-default', (608, 608))
#dt_plate   = ObjectDetector('yolov4-plate_type', (608, 608))
dt_number  = ObjectDetector('yolov4-plate_number', (320, 320))
dt_corner  = CornerDetector('corner_prediction')

<yolov4-plate_number> model loaded.
<corner_prediction> model loaded.


In [3]:
def start(path):
    d_number = None
    img_plate, img_plate_warped = None, None
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

    s_time = time()

    img_plate = dt_corner.loadImage(img)
    img_plate_warped, _ = dt_corner.runDetection()
    
    _ = dt_number.loadImage(img_plate_warped)
    bbox3, obj3 = dt_number.runDetection('number')

    if len(bbox3) != 0:
        nb = [obj3[i] for i in bbox3[:, 0, 0].argsort()]
        d_number = ''
        for n in nb:
            d_number += n

    t_time = time() - s_time

    return t_time, d_number, img_plate, img_plate_warped

def error_log(p, label, predict, write=False, img1=None, img2=None):
    log = {'path': p, 'label': label, 'predict': predict}
    
    if write:
        dir = 'error_log/number_error_AOLP'
        file = osp.basename(p).split('.')[0]
        if not osp.exists(dir): os.mkdir(dir)
        cv2.imwrite(osp.join(dir, file+'-plate.jpg'), cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
        cv2.imwrite(osp.join(dir, file+'-plate_warped.jpg'), cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

    return log

In [4]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\test_data'
plate_corner_dataset = osp.join(dataset_dir, 'plate_corner')
plate_dataset = osp.join(dataset_dir, 'plate_images')

test_path = [str(p) for p in Path(plate_corner_dataset).glob('**/*.jpg')]
print('total:', len(test_path))

total: 2049


In [7]:
acc_list = [0, 0]
num_error = []
total_time = 0
times = 0

for i in tqdm(range(len(test_path)), unit=' images', ncols=100):
    p = test_path[i]
    if i == 0:
        _, _, _, _ = start(p) # skip initial time
    t_time, d_number, img_plate, img_plate_warped = start(p)

    if d_number:
        total_time += t_time
        times += 1
        
        number_file = osp.join(plate_dataset, osp.basename(p).split('-')[0]+'-plate.json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']]
            number = ''
            for n in nb:
                number += n

        acc_list[0] += 1
        if d_number == number:
            acc_list[1] += 1
        else:
            num_error.append(error_log(p, number, d_number, True, img_plate, img_plate_warped))

print('Average time: {:.3f}s'.format(total_time/times))

print('total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))
print('number error:', len(num_error))
with open('error_log/number_error_AOLP.json', 'w') as f:
    json.dump(num_error, f, indent=2)

100%|██████████████████████████████████████████████████████| 2049/2049 [03:26<00:00,  9.92 images/s]

Average time: 0.095s
total=2049, correct=2016, accuracy=98.39%
number error: 33


In [5]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\test_data'
plate_corner_dataset = osp.join(dataset_dir, 'plate_corner')
plate_dataset = osp.join(dataset_dir, 'plate_images')

test_path = [str(p) for p in Path(plate_corner_dataset).glob('**/*.jpg') if 'AC' in osp.basename(str(p)).split('-')[0]]
print('total:', len(test_path))

total: 681


In [6]:
acc_list = [0, 0]
num_error = []
total_time = 0
times = 0

for i in tqdm(range(len(test_path)), unit=' images', ncols=100):
    p = test_path[i]
    if i == 0:
        _, _, _, _ = start(p) # skip initial time
    t_time, d_number, _, _ = start(p)

    if d_number:
        total_time += t_time
        times += 1
        
        number_file = osp.join(plate_dataset, osp.basename(p).split('-')[0]+'-plate.json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']]
            number = ''
            for n in nb:
                number += n

        acc_list[0] += 1
        if d_number == number:
            acc_list[1] += 1
        else:
            num_error.append(error_log(p, number, d_number))

print('Average time: {:.3f}s'.format(total_time/times))
print('total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))

print('number error:', len(num_error))
with open('error_log/number_error_AOLP-AC.json', 'w') as f:
    json.dump(num_error, f, indent=2)

100%|████████████████████████████████████████████████████████| 681/681 [01:13<00:00,  9.29 images/s]

Average time: 0.099s
total= 681, correct= 664, accuracy=97.50%
number error: 17


In [7]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\test_data'
plate_corner_dataset = osp.join(dataset_dir, 'plate_corner')
plate_dataset = osp.join(dataset_dir, 'plate_images')

test_path = [str(p) for p in Path(plate_corner_dataset).glob('**/*.jpg') if 'LE' in osp.basename(str(p)).split('-')[0]]
print('total:', len(test_path))

total: 757


In [8]:
acc_list = [0, 0]
num_error = []
total_time = 0
times = 0

for i in tqdm(range(len(test_path)), unit=' images', ncols=100):
    p = test_path[i]
    if i == 0:
        _, _, _, _ = start(p) # skip initial time
    t_time, d_number, _, _ = start(p)

    if d_number:
        total_time += t_time
        times += 1
        
        number_file = osp.join(plate_dataset, osp.basename(p).split('-')[0]+'-plate.json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']]
            number = ''
            for n in nb:
                number += n

        acc_list[0] += 1
        if d_number == number:
            acc_list[1] += 1
        else:
            num_error.append(error_log(p, number, d_number))

print('Average time: {:.3f}s'.format(total_time/times))
print('total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))

print('number error:', len(num_error))
with open('error_log/number_error_AOLP-LE.json', 'w') as f:
    json.dump(num_error, f, indent=2)

100%|████████████████████████████████████████████████████████| 757/757 [01:15<00:00, 10.00 images/s]

Average time: 0.098s
total= 757, correct= 754, accuracy=99.60%
number error: 3


In [9]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\test_data'
plate_corner_dataset = osp.join(dataset_dir, 'plate_corner')
plate_dataset = osp.join(dataset_dir, 'plate_images')

test_path = [str(p) for p in Path(plate_corner_dataset).glob('**/*.jpg') if 'RP' in osp.basename(str(p)).split('-')[0]]
print('total:', len(test_path))

total: 611


In [10]:
acc_list = [0, 0]
num_error = []
total_time = 0
times = 0

for i in tqdm(range(len(test_path)), unit=' images', ncols=100):
    p = test_path[i]
    if i == 0:
        _, _, _, _ = start(p) # skip initial time
    t_time, d_number, _, _ = start(p)

    if d_number:
        total_time += t_time
        times += 1
        
        number_file = osp.join(plate_dataset, osp.basename(p).split('-')[0]+'-plate.json')
        with open(number_file, 'r') as f:
            nb = [sh['label'] for sh in json.load(f)['shapes']]
            number = ''
            for n in nb:
                number += n

        acc_list[0] += 1
        if d_number == number:
            acc_list[1] += 1
        else:
            num_error.append(error_log(p, number, d_number))

print('Average time: {:.3f}s'.format(total_time/times))
print('total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))

print('number error:', len(num_error))
with open('error_log/number_error_AOLP-RP.json', 'w') as f:
    json.dump(num_error, f, indent=2)

100%|████████████████████████████████████████████████████████| 611/611 [01:00<00:00, 10.13 images/s]

Average time: 0.097s
total= 611, correct= 598, accuracy=97.87%
number error: 13
